In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show, output_file, save, push_notebook, curdoc
from bokeh.plotting import figure, curdoc, gridplot
from bokeh.models import (DateRangeSlider, ColumnDataSource, HoverTool, CustomJS, 
                          Slider, Range1d, FactorRange, Legend, Label, 
                          LabelSet, ColorBar, NumeralTickFormatter, 
                          DatetimeTickFormatter, Toggle, CheckboxGroup, 
                          RadioButtonGroup, TextInput, Button, Div, Tabs)
from bokeh.layouts import row, column, gridplot, layout
from bokeh.palettes import HighContrast3, Viridis256, Category20, Inferno256, Cividis256, Turbo256
from bokeh.transform import factor_cmap, dodge, linear_cmap, log_cmap
from bokeh.events import DoubleTap, MouseMove, PanStart, Tap
# from bokeh.tile_providers import get_provider, Vendors
from bokeh.embed import components, file_html
from bokeh.resources import CDN
from bokeh.themes import Theme
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, HoverTool, RangeTool, Range1d, BoxAnnotation

In [2]:
age_opts = ['6m', '9m', '12m', '18m']
loc_opts = ['STL', 'UNC', 'PHI']
clusters = [1, 2, 3]

x = np.linspace(1, 1000, 1000)
y = np.random.normal(5, 1, len(x))
c = np.random.choice(clusters, size = len(x))
age = np.random.choice(age_opts, size = len(x))
loc = np.random.choice(loc_opts, size = len(x))

data = pd.DataFrame({
    'x': x,
    'y': y,
    'c': c,
    'age' : age, 
    'loc' : loc
}, index=pd.Index(range(1, len(x) + 1), name='frame'))

In [3]:
# Set up notebook output
output_notebook()

# Create a ColumnDataSource
source = ColumnDataSource(data)  # Reset index to include it as a column

# Define colors based on clusters
colors = Category10[3]  # Use a color palette for up to 10 clusters
data['color'] = data['c'].map(lambda cluster: colors[cluster-1])
source.add(data['color'], 'color')

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)

# Add scatter plot to the detailed plot
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)

# Add hover tool
hover = HoverTool(tooltips=TOOLTIPS)
detailed_plot.add_tools(hover)

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)

# Add scatter plot to the minimap
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3

minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)

minimap.add_layout(box)

# Combine detailed plot and minimap
layout = column(detailed_plot, minimap)

# Show the results
show(layout)


Loading BokehJS ...

In [7]:
output_notebook()

# Prepare data for donut plot
cluster_counts = data['c'].value_counts()
clusters = sorted(cluster_counts.index)  # Ensure clusters are sorted or use sorted data
sizes = cluster_counts.loc[clusters].values  # Ensure sizes match the sorted clusters

# Create the angles for the wedges
angles = np.linspace(0, 2 * np.pi, len(clusters) + 1)
start_angles = angles[:-1]
end_angles = angles[1:]

# Create a DataFrame for the wedges
wedge_data = pd.DataFrame({
    'start_angle': start_angles,
    'end_angle': end_angles,
    'color': Category20[len(clusters)],
    'c': clusters,
    'value': sizes
})

# Create a figure
p = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover', tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))

# Draw the donut plot
p.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
        line_color="white", fill_color='color', legend_field='c', source=wedge_data)

# Overlay a white circle to create the "donut" effect
p.circle(x=0, y=0, radius=0.4, fill_color='white')

# Set up the plot
p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

# Show the plot
show(p)

Loading BokehJS ...

In [8]:
output_notebook()

# Prepare data for donut plot
age_counts = data['age'].value_counts()
ages = sorted(age_counts.index)  # Ensure ages are sorted
sizes = age_counts.loc[ages].values  # Ensure sizes match the sorted ages

# Create the angles for the wedges
angles = np.linspace(0, 2 * np.pi, len(ages) + 1)
start_angles = angles[:-1]
end_angles = angles[1:]

# Create a DataFrame for the wedges
wedge_data = pd.DataFrame({
    'start_angle': start_angles,
    'end_angle': end_angles,
    'color': Category20[len(ages)],
    'age': ages,
    'value': sizes
})

# Create a figure
p = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover', tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))

# Draw the donut plot
p.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
        line_color="white", fill_color='color', legend_field='age', source=wedge_data)

# Overlay a white circle to create the "donut" effect
p.circle(x=0, y=0, radius=0.4, fill_color='white')

# Set up the plot
p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

# Show the plot
show(p)

Loading BokehJS ...

In [9]:
output_notebook()

# Prepare data for donut plots
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

# Donut Plot for Clusters
cluster_colors = Category10[3]  # Use a color palette for up to 10 clusters
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover', tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover', tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define colors based on clusters
source.add(data['color'], 'color')

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
hover = HoverTool(tooltips=TOOLTIPS)
detailed_plot.add_tools(hover)

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a placeholder for the image
from bokeh.models import Div
image_placeholder = Div(text='<img src="https://via.placeholder.com/150" alt="Image" width="150" height="150">', width=150, height=150)

# Arrange the plots and image placeholder in a grid layout
layout = gridplot([[cluster_plot, age_plot, image_placeholder], [detailed_plot, minimap]])

# Show the results
show(layout)

Loading BokehJS ...

In [10]:
output_notebook()

# Prepare data for donut plots
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

# Donut Plot for Clusters
cluster_colors = Category10[3]  # Use a color palette for up to 10 clusters
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover', tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover', tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define colors based on clusters
source.add(data['color'], 'color')

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
hover = HoverTool(tooltips=TOOLTIPS)
detailed_plot.add_tools(hover)

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a placeholder for the image
image_placeholder = Div(text='<img src="https://via.placeholder.com/150" alt="Image" width="150" height="150">', width=150, height=150)

# Arrange the plots and image placeholder in a grid layout with minimap below the detailed plot
layout = gridplot([[cluster_plot, age_plot, image_placeholder], [detailed_plot], [minimap]])

# Show the results
show(layout)

Loading BokehJS ...

In [11]:
output_notebook()

# Prepare data for donut plots
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

# Donut Plot for Clusters
cluster_colors = Category10[3]  # Use a color palette for up to 10 clusters
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover', tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover', tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define colors based on clusters
source.add(data['color'], 'color')

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
hover = HoverTool(tooltips=TOOLTIPS)
detailed_plot.add_tools(hover)

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a button to toggle the minimap
toggle_button = Button(label="Toggle Minimap", button_type="success")

# JavaScript callback to toggle minimap visibility
toggle_button.js_on_click(CustomJS(args=dict(minimap=minimap), code="""
    minimap.visible = !minimap.visible;
"""))

# Create a placeholder for the image
image_placeholder = Div(text='<img src="https://via.placeholder.com/150" alt="Image" width="150" height="150">', width=150, height=150)

# Arrange the plots in a grid layout
plot_layout = gridplot([[cluster_plot, age_plot], [detailed_plot], [minimap]])

# Combine the button, image, and plots
layout = column(toggle_button, image_placeholder, plot_layout)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)

Loading BokehJS ...

In [14]:
output_notebook()

# Prepare data for donut plots
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover', tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover', tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define colors based on clusters
source.add(data['color'], 'color')

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
hover = HoverTool(tooltips=TOOLTIPS)
detailed_plot.add_tools(hover)

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a button to toggle the minimap
toggle_button = Button(label="Toggle Minimap", button_type="success")

# JavaScript callback to toggle minimap visibility
toggle_button.js_on_click(CustomJS(args=dict(minimap=minimap), code="""
    minimap.visible = !minimap.visible;
"""))

# Create a placeholder for the image
image_placeholder = Div(text='<img src="https://via.placeholder.com/150" alt="Image" width="150" height="150">', width=150, height=150)

# Add a blurb of text with HTML and LaTeX
text_blurb = Div(text="""
    <h2>Data Visualization Dashboard</h2>
    <p>This dashboard includes various visualizations to help understand the dataset:</p>
    <ul>
        <li>Donut plots representing <b>clusters</b> and <b>age groups</b>.</li>
        <li>A detailed scatter plot with an interactive minimap.</li>
    </ul>
    <p>Mathematical formula example: \( E = mc^2 \)</p>
""", width=800)

# Arrange the plots and other elements in a grid layout
plot_layout = gridplot([[cluster_plot], [age_plot], [detailed_plot], [minimap]])

# Combine the button, text blurb, image, and plots
layout = column(text_blurb, toggle_button, image_placeholder, plot_layout)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)

<>:117: SyntaxWarning: invalid escape sequence '\('
<>:117: SyntaxWarning: invalid escape sequence '\('
/var/folders/pf/49k8jg7948ddty_lg9b4pybr0000gn/T/ipykernel_69664/3022914445.py:117: SyntaxWarning: invalid escape sequence '\('
  text_blurb = Div(text="""


Loading BokehJS ...